This notebook is to run NMM with `neurolib` pkg

test its running time

In [1]:
import sys
sys.path.append("../../../mypkg")

import scipy
import itertools

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import trange
from functools import partial
from easydict import EasyDict as edict
from collections import defaultdict as ddict
from IPython.display import display
from scipy import signal

In [2]:
# the main pkg to implemente the NMM
import neurolib
from neurolib.models.wc import WCModel
from neurolib.optimize.evolution import Evolution
from neurolib.utils.signal import Signal 
from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.utils.loadData import Dataset
import neurolib.utils.functions as func

# to calculate coh FC
import mne
from mne_connectivity import spectral_connectivity_epochs

In [3]:
import logging
logging.getLogger("root").setLevel(logging.WARNING)
logging.getLogger("mne").setLevel(logging.WARNING)
logging.getLogger("pypet").setLevel(logging.WARNING)

In [4]:
# my own fns
from brain import Brain
from constants import RES_ROOT, DATA_ROOT, FIG_ROOT
from utils.misc import load_pkl, save_pkl
from utils.measures import geodesic_dist, reg_R_fn, lin_R_fn, lap_mat_fn

plt.style.use(FIG_ROOT/"base.mplstyle")

# Prepare

## Fns

In [5]:
_minmax_vec = lambda x: (x-np.min(x))/(np.max(x)-np.min(x));
    
def _model2data(wc):
    """Get time series data from the run model"""
    sim_signal = xr.DataArray(wc.exc[:68, int(1000/wc.params.dt):], 
                              dims=("regions", "time"), 
                              coords={"time": wc.t[int(1000/wc.params.dt):]/1000}, 
                              attrs={'atlas':'DK68'});
    sim_signal = Signal(sim_signal);
    sim_signal.resample(to_frequency=600)
    return np.array(sim_signal.data)

def _get_wc(sc, dmat):
    """Get the NMM model with SC and dmat
       Set the default parameters
    """
    wc = WCModel(Cmat = sc, 
                 Dmat = dmat,
                 seed=0)
    wc.params['duration'] = 61*1000 # 60 seconds -- The unit is in 1 ms
    wc.params['dt'] = 1;
    wc.params['K_gl'] = 6.55
    wc.params['exc_ext'] = 1.58
    wc.params['inh_ext'] = 2.83
    wc.params['sigma_ou'] = 0.02
    return wc

def _get_simu_fc(input_signal, paras={}):
    """
    Calculate functional connectivity from input_signal using the parameters in paras.

    Parameters:
    input_signal (numpy.ndarray): Input signal with shape (n_channels, n_timepont)
    paras (dict): Dictionary containing the following keys and values:
        - nepoch (int): Number of epochs
        - fc_type (str): Functional connectivity method
        - f_skip (int): Frequency skip
        - bd (str): Frequency band
        - faverage (bool): If True, average the connectivity over frequency bands

    Returns:
    numpy.ndarray: Functional connectivity matrix
    """
    def __2matf(mat):
        mat_f = mat + mat.T
        mat_f = mat_f - np.diag(np.diag(mat))
        return mat_f

    _bd_limits = edict()
    _bd_limits.delta = [2, 3.5]
    _bd_limits.theta = [4, 7]
    _bd_limits.alpha = [8, 12]
    _bd_limits.beta_l = [13, 20]
    _fs = 600 # sampling freq

    _num_taps = np.array([551, 351, 351, 351, 251, 251]);
    _bd_order = ['delta' , 'theta', 'alpha' , 'beta', 'beta_l', 'beta_h'] 

    _paras = edict()
    _paras.bd = "alpha"
    _paras.fc_type = "coh"
    _paras.f_skip = 10
    _paras.nepoch = 100
    _paras.update(paras)
    
    input_signal = signal.detrend(input_signal, axis=1, type="linear", bp=0, overwrite_data=False);
    bii = signal.firwin(_num_taps[_bd_order.index(_paras.bd)], 
                       _bd_limits[_paras.bd], 
                       pass_zero=False, 
                       fs=_fs, 
                       window="hamming");


    input_signal = signal.filtfilt(bii, 1, input_signal, axis=1);
    
    if _paras.nepoch == 1:
        input_signal = input_signal[np.newaxis]
    else:
        input_signal = input_signal.reshape(68, _paras.nepoch, -1).transpose(1, 0, 2)
    
    ts_con = spectral_connectivity_epochs(input_signal,
                                          names=None, 
                                          method=_paras.fc_type, 
                                          indices=None, 
                                          sfreq=_fs, 
                                          mode='multitaper',
                                          fmin=None, 
                                          fmax=np.inf,
                                          fskip=_paras.f_skip, 
                                          faverage=True, 
                                          tmin=None, 
                                          tmax=None,  
                                          mt_bandwidth=None, 
                                          mt_adaptive=False, 
                                          mt_low_bias=True, 
                                          cwt_freqs=None, 
                                          cwt_n_cycles=7, 
                                          block_size=1000, 
                                          n_jobs=1, 
                                          verbose=False)
    mat = ts_con.get_data(output='dense').squeeze();
    return __2matf(mat)

In [6]:
def _add_v2con(cur_ind_conn, add_v=0.01):
    cur_ind_conn = cur_ind_conn.copy()
    add_v = np.quantile(cur_ind_conn, 0.99)*add_v # tuning 0.1
    np.fill_diagonal(cur_ind_conn[:34, 34:68], np.diag(cur_ind_conn[:34, 34:68]) + add_v)
    np.fill_diagonal(cur_ind_conn[34:68, :34], np.diag(cur_ind_conn[34:68, :34]) + add_v)
    np.fill_diagonal(cur_ind_conn[68:77, 77:], np.diag(cur_ind_conn[68:77, 77:]) + add_v)
    np.fill_diagonal(cur_ind_conn[77:, 68:77], np.diag(cur_ind_conn[77:, 68:77]) + add_v)
    return cur_ind_conn

## Param

In [7]:
bds = ["delta", "theta", "alpha", "beta_l"]
paras = edict()
paras.uptri_idxs = np.triu_indices(68, k=1);
paras.add_v = 0.01
paras.nepoch = 100
paras.loss_type = "linr"
paras.cur_bd = "alpha"

if paras.loss_type == "linr":
    def _loss_fn(x, y):
        return -lin_R_fn(x, y)[0]
elif paras.loss_type == "mse":
    def _loss_fn(x, y):
        return np.mean((x-y)**2)

## Load data

In [8]:
# A fun to load emp FC
fc_root = RES_ROOT/"emp_fcs2"
def _get_emp_fc(sub_ix, bd):
    fil = list(fc_root.rglob(f"*{bd}*{paras.nepoch}/sub{sub_ix}.pkl"))[0]
    return np.abs(load_pkl(fil, verbose=False))

In [9]:
# load SC and dmat, and do some preprocessing
# SC
ind_conn_xr = xr.open_dataarray(DATA_ROOT/'individual_connectomes_reordered.nc')
ind_conn = ind_conn_xr.values;

scs = []
for cur_ind_idx in range(36):
    # create spectrome brain:
    brain = Brain.Brain()
    brain.add_connectome(DATA_ROOT) # grabs distance matrix
    # re-ordering for DK atlas and normalizing the connectomes:
    brain.reorder_connectome(brain.connectome, brain.distance_matrix)
    # re-assign connectome to individual connectome
    brain.connectome =  _add_v2con(ind_conn[:, :, cur_ind_idx], add_v=paras.add_v)
    brain.bi_symmetric_c()
    brain.reduce_extreme_dir()
    sc = brain.reducedConnectome
    scs.append(sc[:, :])
scs = np.array(scs)
dmat = brain.distance_matrix[:, :];

# based on the paper, we should normalize it (Cakan_et_al_CC_2023_neurolib_pythonpkg_NMM.pdf)
scs_norm = scs/scs.max(axis=(1, 2), keepdims=1);

# NMM

In [10]:
# the parameters space, from 
# https://neurolib-dev.github.io/examples/example-3-meg-functional-connectivity/#model-fit
pars = ParameterSpace(['K_gl', 'exc_ext', 'inh_ext', 'sigma_ou'], 
                      [[0.0, 20.0], [0.0, 4.0], [0.0, 4.0], [0.001, 0.5]]);

In [11]:
# the core fn to run optimization
def evaluate_simulation(traj):
    rid = traj.id
    model = evolution.getModelFromTraj(traj)

    # -------- simulation --------

    model.run()

    # -------- fitness evaluation here --------
    
    simulated_data = _model2data(model);
    
    simulated_fc = _cur_get_simu_fc(simulated_data);
    simulated_vec = _minmax_vec(simulated_fc[tuple(paras.uptri_idxs)]);
    emp_vec = _minmax_vec(cur_fc[tuple(paras.uptri_idxs)]);
    score = _loss_fn(simulated_vec, emp_vec)
    # the output
    results = {
        "simulated_fc": simulated_fc
    }

    return (score,), results

## Run

In [13]:
import time
cur_ind_idx = 1
ts = [time.time()]
for cur_ind_idx in range(10):
    print(f"It is subject {cur_ind_idx:.0f} of the {paras.cur_bd} band!")
    _cur_get_simu_fc = partial(_get_simu_fc, paras={"bd":paras.cur_bd, 
                                                   "nepoch":paras.nepoch})
    cur_fc = _get_emp_fc(cur_ind_idx, paras.cur_bd)
    cur_sc = scs_norm[cur_ind_idx]
    
    cur_model = _get_wc(cur_sc, dmat)
    evolution = Evolution(evaluate_simulation, pars, 
                          algorithm = 'nsga2', 
                          weightList = [-1.0], model = cur_model, 
                          POP_INIT_SIZE=128, POP_SIZE=64, NGEN=16, 
                          filename=RES_ROOT/f"tmp/ind{cur_ind_idx}.hdf")
    
    evolution.run(verbose=True, verbose_plotting=False)
    ts.append([time.time()])

It is subject 0 of the alpha band!


MainProcess root INFO     Evolution: Using algorithm: nsga2
MainProcess root INFO     Evolution: Individual generation: <function randomParameters at 0x7f74b90173a0>
MainProcess root INFO     Evolution: Mating operator: <function cxSimulatedBinaryBounded at 0x7f74b94dc670>
MainProcess root INFO     Evolution: Mutation operator: <function mutPolynomialBounded at 0x7f74b946d670>
MainProcess root INFO     Evolution: Parent selection: <function selTournamentDCD at 0x7f74b94dcdc0>
MainProcess root INFO     Evolution: Selection operator: <function selNSGA2 at 0x7f74b94dc9d0>
MainProcess root INFO     Evaluating initial population of size 128 ...


Parameter distribution (Generation 0):
K_gl: 	 mean: 9.6009,	 std: 5.5431
exc_ext: 	 mean: 1.9502,	 std: 1.1711
inh_ext: 	 mean: 2.0382,	 std: 1.1531
sigma_ou: 	 mean: 0.2482,	 std: 0.1561


MainProcess root INFO     Start of evolution
MainProcess root INFO     Replacing 0 invalid individuals.
MainProcess root INFO     Replacing 0 invalid individuals.


----------- Generation 1 -----------
Best individual is [1.2336056386725436, 0.2747712475268718, 2.4131034684302537, 0.147912270838099]
Score: 0.22292916450167696
Fitness: (-0.22292916450167696,)
--- Population statistics ---
> Simulation parameters
HDF file storage: /data/rajlab1/user_data/jin/MyResearch/SBI-SGM-FC/notebooks/new_sgm/exe_times/../../../mypkg/../results/tmp/ind0.hdf
Trajectory Name: results-2023-12-19-18H-40M-55S
Duration of evaluating initial population 0:03:27.975550
Model: <class 'neurolib.models.wc.model.WCModel'>
Model name: wc
Eval function: <function evaluate_simulation at 0x7f74b86f2670>
Parameter space: {'K_gl': [0.0, 20.0], 'exc_ext': [0.0, 4.0], 'inh_ext': [0.0, 4.0], 'sigma_ou': [0.001, 0.5]}
> Evolution parameters
Number of generations: 16
Initial population size: 128
Population size: 64
> Evolutionary operators
Mating operator: <function cxSimulatedBinaryBounded at 0x7f74b94dc670>
Mating paramter: {'low': [0.0, 0.0, 0.0, 0.001], 'up': [20.0, 4.0, 4.0, 0.5]

...

----------- Generation 13 -----------
Best individual is [1.0165335190867393, 0.5244222297479272, 2.210008468787415, 0.10833509736501491]
Score: 0.2381285800760252
Fitness: (-0.2381285800760252,)
--- Population statistics ---
> Simulation parameters
HDF file storage: /data/rajlab1/user_data/jin/MyResearch/SBI-SGM-FC/notebooks/new_sgm/exe_times/../../../mypkg/../results/tmp/ind9.hdf
Trajectory Name: results-2023-12-19-23H-18M-21S
Duration of evaluating initial population 0:03:26.996068
Model: <class 'neurolib.models.wc.model.WCModel'>
Model name: wc
Eval function: <function evaluate_simulation at 0x7f74b86f2670>
Parameter space: {'K_gl': [0.0, 20.0], 'exc_ext': [0.0, 4.0], 'inh_ext': [0.0, 4.0], 'sigma_ou': [0.001, 0.5]}
> Evolution parameters
Number of generations: 16
Initial population size: 128
Population size: 64
> Evolutionary operators
Mating operator: <function cxSimulatedBinaryBounded at 0x7f74b94dc670>
Mating paramter: {'low': [0.0, 0.0, 0.0, 0.001], 'up': [20.0, 4.0, 4.0, 0.5]

MainProcess root INFO     Replacing 0 invalid individuals.


----------- Generation 14 -----------
Best individual is [1.0165335190867393, 0.5244222297479272, 2.210008468787415, 0.10833509736501491]
Score: 0.2381285800760252
Fitness: (-0.2381285800760252,)
--- Population statistics ---
> Simulation parameters
HDF file storage: /data/rajlab1/user_data/jin/MyResearch/SBI-SGM-FC/notebooks/new_sgm/exe_times/../../../mypkg/../results/tmp/ind9.hdf
Trajectory Name: results-2023-12-19-23H-18M-21S
Duration of evaluating initial population 0:03:26.996068
Model: <class 'neurolib.models.wc.model.WCModel'>
Model name: wc
Eval function: <function evaluate_simulation at 0x7f74b86f2670>
Parameter space: {'K_gl': [0.0, 20.0], 'exc_ext': [0.0, 4.0], 'inh_ext': [0.0, 4.0], 'sigma_ou': [0.001, 0.5]}
> Evolution parameters
Number of generations: 16
Initial population size: 128
Population size: 64
> Evolutionary operators
Mating operator: <function cxSimulatedBinaryBounded at 0x7f74b94dc670>
Mating paramter: {'low': [0.0, 0.0, 0.0, 0.001], 'up': [20.0, 4.0, 4.0, 0.5]

MainProcess root INFO     --- End of evolution ---
MainProcess root INFO     Best individual is [1.2807672669387817, 0.3197876405031276, 2.838067567172477, 0.1352089772763401], (-0.2386532092254212,)
MainProcess root INFO     --- End of evolution ---


----------- Generation 15 -----------
Best individual is [1.2807672669387817, 0.3197876405031276, 2.838067567172477, 0.1352089772763401]
Score: 0.2386532092254212
Fitness: (-0.2386532092254212,)
--- Population statistics ---
> Simulation parameters
HDF file storage: /data/rajlab1/user_data/jin/MyResearch/SBI-SGM-FC/notebooks/new_sgm/exe_times/../../../mypkg/../results/tmp/ind9.hdf
Trajectory Name: results-2023-12-19-23H-18M-21S
Duration of evaluating initial population 0:03:26.996068
Model: <class 'neurolib.models.wc.model.WCModel'>
Model name: wc
Eval function: <function evaluate_simulation at 0x7f74b86f2670>
Parameter space: {'K_gl': [0.0, 20.0], 'exc_ext': [0.0, 4.0], 'inh_ext': [0.0, 4.0], 'sigma_ou': [0.001, 0.5]}
> Evolution parameters
Number of generations: 16
Initial population size: 128
Population size: 64
> Evolutionary operators
Mating operator: <function cxSimulatedBinaryBounded at 0x7f74b94dc670>
Mating paramter: {'low': [0.0, 0.0, 0.0, 0.001], 'up': [20.0, 4.0, 4.0, 0.5],

In [17]:
ts1 = [ts[0]] + [i[0] for i in ts[1:]]

[1703040055.485717,
 1703041902.2550979,
 1703043749.7346141,
 1703045596.7435083,
 1703047451.0563412,
 1703049300.4229057,
 1703051150.0903292,
 1703052995.4490316,
 1703054849.1989937,
 1703056701.5331223,
 1703058553.6264756]

In [18]:
dlt_ts = np.diff(ts1)
print(f"The time for each subject is {np.mean(dlt_ts):.3f} with std {np.std(dlt_ts):.3f}.")

The time for each subject is 1849.814 with std 2.999.
